In [9]:
import tensorflow as tf
import numpy as np
import tempfile

In [10]:
batchSize = 5
maxNumSteps = 8
embeddingSize = 3
nNeurons = 4
vocabulary_size = 10
initEmbeddings = np.random.randn(vocabulary_size, embeddingSize)
bias_trainable = False

In [32]:
tf.reset_default_graph()
inputTokens = tf.placeholder(tf.int32, [batchSize, maxNumSteps])
inputLens = tf.placeholder(tf.int32, [batchSize])
targets = tf.placeholder(tf.float64, [batchSize])
rnnCell = tf.nn.rnn_cell.BasicRNNCell(nNeurons)
# initState = rnnCell.zero_state(batchSize, tf.float64)
# initState = tf.get_variable("initState", [batchSize, nNeurons], dtype=tf.float64, trainable=True)
initState = tf.get_variable("initState", [nNeurons], dtype=tf.float64, trainable=True)
initStates = tf.concat(0, [initState for i in xrange(batchSize)])
initStates = tf.reshape(initStates, [-1, nNeurons])

embedding = tf.Variable(initEmbeddings, name='inputEmbeddings', trainable=False, dtype=tf.float64)
inputData = tf.nn.embedding_lookup(embedding, inputTokens)

raw_outputs, last_states = tf.nn.dynamic_rnn(
    cell=rnnCell,
    initial_state=initStates,
    dtype=tf.float64,
    sequence_length=inputLens,
    inputs=inputData)
    
flattened_outputs = tf.reshape(raw_outputs, [-1, nNeurons])
index = tf.range(0, batchSize) * maxNumSteps + inputLens - 1
outputs = tf.gather(flattened_outputs, index)
outputW = tf.get_variable("outputW", [nNeurons, 1], dtype=tf.float64)
outputB = tf.get_variable("outputB", [1], dtype=tf.float64)
prediction = tf.add(tf.matmul(outputs, outputW), outputB)
loss = tf.reduce_sum(tf.pow(prediction-targets, 2)/batchSize)

lr = tf.Variable(0.1, trainable=False)
tvars = tf.trainable_variables()
if not bias_trainable:
    tvars = np.take(tvars, [0,1,3,4]).tolist() # exclude RNN bias from training
optimizer = tf.train.GradientDescentOptimizer(lr)
learningStep = optimizer.minimize(loss, var_list=tvars)
initAll = tf.initialize_all_variables()

In [33]:
# data = np.random.randn(batchSize, maxNumSteps, embeddingSize)
data = np.random.randint(0, high=vocabulary_size-1, size=(batchSize, maxNumSteps))
lens = [maxNumSteps for i in xrange(batchSize)]
t = np.random.randn(batchSize)
with tf.Session() as sess:
    sess.run(initAll)
#     e = sess.run(embedding)
#     print e
#     print data
#     s = sess.run(inputData, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print s
#     l = sess.run(loss, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print l
    for v in tf.trainable_variables():
        print v.name
        print sess.run(v)
#     w = sess.run(outputW)
#     b = sess.run(outputB)
#     print w
#     print b
#     r = sess.run(raw_outputs, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print len(r), len(r[0])
#     print r[:,-1]
#     i = sess.run(index, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print i
#     o = sess.run(outputs, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print o
#     m = sess.run(tf.matmul(outputs, outputW), feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print m
#     p = sess.run(prediction, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print p
    sess.run(learningStep, feed_dict={inputTokens:data, inputLens:lens, targets:t})
    l = sess.run(loss, feed_dict={inputTokens:data, inputLens:lens, targets:t})
    print l
    for v in tf.trainable_variables():
        print v.name
        print sess.run(v)

initState:0
[ 1.46410907 -0.63501613 -0.24634122 -0.29501   ]
RNN/BasicRNNCell/Linear/Matrix:0
[[-0.01703865  0.43892278  0.50730039  0.42165349]
 [ 0.25936399  0.43430144  0.00343578  0.06466562]
 [ 0.43580867  0.61887269 -0.6188308   0.07348994]
 [-0.26377648 -0.41525456 -0.30676125 -0.42555988]
 [-0.53021303  0.36482772  0.03989031 -0.17272379]
 [-0.22105937 -0.24776662 -0.19847863 -0.03338807]
 [ 0.52217622 -0.27809662 -0.46751162  0.19431991]]
RNN/BasicRNNCell/Linear/Bias:0
[ 0.  0.  0.  0.]
outputW:0
[[ 0.80211346]
 [ 0.2293196 ]
 [ 0.70090983]
 [-0.20661571]]
outputB:0
[-1.14146178]
6.18506051651
initState:0
[ 1.47123468 -0.63098721 -0.24048359 -0.29496161]
RNN/BasicRNNCell/Linear/Matrix:0
[[-0.21547607  0.34160228  0.3690477   0.39353471]
 [-0.29171587  0.14842786 -0.56030575  0.07662606]
 [ 0.32727858  0.70402887 -0.98735605  0.03034594]
 [ 0.06416492 -0.2610832   0.01334147 -0.46709646]
 [ 0.10560379  0.58607889  0.71631451 -0.34082643]
 [ 0.18353758 -0.18334824  0.15375016 -

In [29]:
for v in tf.trainable_variables():
    print v.name

initState:0
RNN/BasicRNNCell/Linear/Matrix:0
RNN/BasicRNNCell/Linear/Bias:0
outputW:0
outputB:0


In [37]:
np.dot([[1,2],[3,4]], [[1,2],[2,1]])

array([[ 5,  4],
       [11, 10]])